# Constants:

In [2]:
import warnings
warnings.simplefilter('ignore')

%display latex

mt_voltage = 4
mt_max_current = 0.050

# https://en.wikipedia.org/wiki/American_wire_gauge
ohms_per_meter_32_AWG = 0.5383
ohms_per_meter_39_AWG = 2.729

# Aircore calculations:

All outputs in A m<sup>2</sup>

In [9]:
l, w, N = var( 'l w N' )

# the length and width are positive, and the number of turns is greater than 1.
# Helps with symbolic optimization and debugging.
assume( l > 0 )
assume( w > 0 )
assume( N >= 1 )

# the current is the minimum between Ohm's law on 32 AWG wire and the max output of 50 mA
I( l, w, N ) = min_symbolic( ( mt_voltage / ( ( 2 * l + 2 * w ) * N * ohms_per_meter_32_AWG ) ), mt_max_current )

# Basically, the dipole moment of the aircore is mu = NIA
dipole_aircore( l, w, N ) = N * I(l, w, N) * (l * w)

# Using roughly our known parameters and assuming the magnetorquer as square:
dipole_aircore(
    l=0.055,
    w=0.062,
    N=820 ).n(20)

0.10829

# Ferrocore Calculations:

In [7]:
l, r, N = var( 'l r N' )
u_r = var( 'u_r', latex_name="\mu_r" ) 
N_d = var( 'N_d', latex_name="N_d" )

# again, the length and radius are greater than 0 and the number of turns is more than 1
assume( l > 0 )
assume( r > 0 )
assume( N >= 1 )

# Demagnetization and amplification formulae used from:
# http://www.aerospades.com/uploads/3/7/3/2/37325123/cadre_torquers.pdf
demagnetization( l, r ) = 4 * ( ln( l / r ) - r ) / ( ( l / r )^2 - 4 * ln( l / r ) )

amplification( r, l, u_r ) = 1 + ( u_r - 1 ) / ( 1 + ( u_r - 1 ) * demagnetization( l, r ) )

# Current is the minimum of the resistance of the wire and the max current
current_ferrocore( r, N ) = min_symbolic( mt_voltage / ( 2 * pi * r * N / ohms_per_meter_39_AWG ), mt_max_current )

# u_r = 15000
# http://www.edfagan.com/hiperco-50-coil.php

dipole_ferrocore( r, l, N, u_r ) = pi * r^2 * N * current_ferrocore( N, r ) * amplification( r, l, u_r )
dipole_ferrocore( 
    r=0.0035, 
    l=0.070, 
    N=1500,
    u_r=15000 ).n(20)

0.096256